# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('../previsao_de_renda.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               12427 non-null  object 
 1   id_cliente             12427 non-null  int64  
 2   sexo                   12427 non-null  object 
 3   posse_de_veiculo       12427 non-null  bool   
 4   posse_de_imovel        12427 non-null  bool   
 5   qtd_filhos             12427 non-null  int64  
 6   tipo_renda             12427 non-null  object 
 7   educacao               12427 non-null  object 
 8   estado_civil           12427 non-null  object 
 9   tipo_residencia        12427 non-null  object 
 10  idade                  12427 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  12427 non-null  float64
 13  renda                  12427 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usage: 1

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [23]:
data = df.copy()
var_qualitativas = data.loc[:, data.dtypes == 'object'].columns
var_qualitativas

Index(['data_ref', 'sexo', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia'],
      dtype='object')

In [81]:
def freq(series):
    bigger = 0
    current_index = ''
    for i, value in enumerate(series.value_counts()):
        if i == 0:
            bigger = value / series.count()
            current_index = series.value_counts().index
        if value / series.count() > bigger:
            bigger = value / series.count()
            current_index = series.index
    return bigger, current_index

In [82]:
freq(data['data_ref'])

(0.06864086263780478,
 Index(['2015-09-01', '2015-05-01', '2015-07-01', '2016-02-01', '2015-01-01',
        '2016-01-01', '2015-10-01', '2015-11-01', '2015-04-01', '2015-02-01',
        '2015-06-01', '2015-08-01', '2015-03-01', '2015-12-01', '2016-03-01'],
       dtype='object', name='data_ref'))

In [67]:
d = data['data_ref'].value_counts()
[index if d[index] == 853 else 0 for index in d.index]

['2015-09-01', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
y, X = patsy.dmatrices('np.log(renda) ~ C(data_ref) + C(sexo) + tipo_renda + educacao + estado_civil + tipo_residencia',
                      data)
reg = sm.OLS(y, X).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     63.47
Date:                Mon, 24 Jun 2024   Prob (F-statistic):               0.00
Time:                        20:52:38   Log-Likelihood:                -15373.
No. Observations:               12427   AIC:                         3.081e+04
Df Residuals:                   12394   BIC:                         3.106e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            7.7641      0.108     72.196      0.000       7.553       7.975
C(data_ref)[T.2015-02-01]           -0.0119      0.041     -0.291      0.771      -0.092       0.068
C(data_ref)[T.2015-03-01]            0.0322      0.041      0.784      0.433      -0.048       0.113
C(data_ref)[T.2015-04-01]            0.0646      0.041      1.575      0.115      -0.016       0.145
C(data_ref)[T.2015-05-01]           -0.0366      0.041     -0.895      0.371      -0.117       0.044
C(data_ref)[T.2015-06-01]            0.0695      0.041      1.693      0.090      -0.011       0.150
C(data_ref)[T.2015-07-01]            0.0170      0.041      0.415      0.678      -0.063       0.097
C(data_ref)[T.2015-08-01]            0.0019      0.041      0.045      0.964      -0.079       0.082
C(data_ref)[T.2015-09-01]           -0.0150      0.041     -0.368      0.713      -0.095       0.065
C(data_ref)[T.2015-10-01]            0.0260      0.041      0.634      0.526      -0.054       0.106
C(data_ref)[T.2015-11-01]            0.0170      0.041      0.414      0.679      -0.063       0.098
C(data_ref)[T.2015-12-01]            0.0676      0.041      1.640      0.101      -0.013       0.148
C(data_ref)[T.2016-01-01]           -0.0244      0.041     -0.594      0.553      -0.105       0.056
C(data_ref)[T.2016-02-01]            0.0171      0.041      0.417      0.676      -0.063       0.098
C(data_ref)[T.2016-03-01]           -0.0511      0.041     -1.235      0.217      -0.132       0.030
C(sexo)[T.M]                         0.6842      0.016     43.013      0.000       0.653       0.715
tipo_renda[T.Bolsista]               0.4057      0.279      1.455      0.146      -0.141       0.952
tipo_renda[T.Empresário]             0.0708      0.017      4.117      0.000       0.037       0.104
tipo_renda[T.Pensionista]           -0.3881      0.279     -1.390      0.164      -0.935       0.159
tipo_renda[T.Servidor público]       0.2156      0.026      8.438      0.000       0.166       0.266
educacao[T.Pós graduação]           -0.0139      0.184     -0.075      0.940      -0.374       0.346
educacao[T.Secundário]               0.0676      0.083      0.812      0.417      -0.096       0.231
educacao[T.Superior completo]        0.1492      0.084      1.785      0.074      -0.015       0.313
educacao[T.Superior incompleto]     -0.0611      0.090     -0.679      0.497      -0.237       0.115
estado_civil[T.Separado]             0.0677      0.032      2.105      0.035       0.005       0.131
estado_civil[T.Solteiro]            -0.0331      0.023     -1.427      0.154      -0.079       0.012
estado_civil[T.União]               -0.0724      0.029     -2.496      0.013      -0.129      -0.016
estado_civil[T.Viúvo]                0.1315      0.04